In [1]:
import nltk
import numpy as np
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\voilentKiller0\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\voilentKiller0\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

Text Paragraph

In [2]:
plag = "The legal system is made up of civil courts, criminal courts and specialty courts, such as family law courts and bankruptcy courts. Each court has its own jurisdiction, which refers to the cases that the court is allowed to hear. In some instances, a case can only be heard in one type of court. For example, a bankruptcy case must be heard in a bankruptcy court. In other instances, more than one court could potentially have jurisdiction. For example, a federal criminal court and a state criminal court would each have jurisdiction over a crime that is a federal drug offense but that is also a state offense."
original = "The legal system is made up of criminal and civil courts and specialty courts like bankruptcy and family law courts. Each court is vested with its own jurisdiction. Jurisdiction refers to the types of cases the court is permitted to rule on. Sometimes, only one type of court can hear a particular case. For instance, bankruptcy cases can be ruled on only in bankruptcy court. In other situations, it is possible for more than one court to have jurisdiction. For instance, both a state and federal criminal court could have authority over a criminal case that is also considered an offense under federal and state drug laws."
target ="Gardening in mixed beds is a great way to get the most productivity from a small space. Some investment is required, to purchase materials for the beds themselves, as well as soil and compost. The investment will likely pay-off in terms of increased productivity."
references ="If you don’t have a lot of space for a garden, raised beds can be a great option. Gardening in mixed beds is a great way to get the most productivity from a small area. Some investment is required. You’ll need to purchase materials for the raised beds themselves, as well as soil and compost. The investment will pay off, though, in the form of increased productivity."

In [5]:
def findNouns(para):
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = nltk.word_tokenize(para)
    nouns = [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)]
    return nouns 

In [6]:
target_noun, ref_noun=findNouns(target),findNouns(references)

In [11]:
print (f"Nouns list in target paragraph : {target_noun}\n\nNouns list in reference paragraph : {ref_noun}")

Nouns list in target paragraph : ['beds', 'way', 'productivity', 'space', 'investment', 'materials', 'beds', 'soil', 'compost', 'investment', 'terms', 'productivity']

Nouns list in reference paragraph : ['t', 'lot', 'space', 'garden', 'beds', 'option', 'Gardening', 'beds', 'way', 'productivity', 'area', 'investment', 'need', 'materials', 'beds', 'soil', 'compost', 'investment', 'form', 'productivity']


In [12]:
def sharedWords(list1,list2):
    list1s = set(list1)
    shared = list1s.intersection(list2)
    return shared 

In [13]:
shared = sharedWords(target_noun, ref_noun)
print (f"Shared nouns word between two paragreph target and reference : {shared}")

Shared nouns word between two paragreph target and reference : {'space', 'productivity', 'beds', 'investment', 'materials', 'soil', 'compost', 'way'}


In [14]:
def findsharedAmount(list1,list2):
    m=0
    for i in list1:
        if i in list2:
            m += len(i)
    return m

In [15]:
sharedAmtTarget, sharedAmtRef = findsharedAmount(target_noun, shared), findsharedAmount(ref_noun, shared)
print (f"Shared amount in target noun : {sharedAmtTarget}\nShared amount in reference noun : {sharedAmtRef}")

Shared amount in target noun : 80
Shared amount in reference noun : 84


In [16]:
def thres(nt,nr,M):
    x=0
    y=0
    for i in nt:
        x+=len(i)
    for i in nr:
        y+=len(i)
    return min(x//M,y//M)

In [17]:
def defthreshold(C):
    m=0
    for i in C:
        m+=len(i)
    return m

In [18]:
def generateflags(Ft,nt,C):
    for i in range(len(nt)):
        if nt[i] in C:
            Ft[i]=True
    return Ft

In [19]:
def findIsolatedFalse(nt,Ft):
    for i in range(len(Ft)-2):
        x=Ft[i:i+3]
        if x[1]==False and x[0] ==True and x[2]==True:
            x[1]=True
            Ft[i:i+3] = x
            nt[i+1]='#'
    return nt,Ft

In [21]:
falseTarget = [False]*len(target_noun)
falseRef = [False]*len(ref_noun)
falseTarget = generateflags(falseTarget,target_noun,shared)
falseRef = generateflags(falseRef,ref_noun,shared)


In [22]:
print (falseTarget,falseRef, sep = "\n")

[True, True, True, True, True, True, True, True, True, True, False, True]
[False, False, True, False, True, False, False, True, True, True, False, True, False, True, True, True, True, True, False, True]


In [23]:
target_noun, falseTarget = findIsolatedFalse(target_noun, falseTarget)
ref_noun, falseRef = findIsolatedFalse(ref_noun, falseRef)

In [26]:
print (target_noun,falseTarget, sep = "\n")

['beds', 'way', 'productivity', 'space', 'investment', 'materials', 'beds', 'soil', 'compost', 'investment', '#', 'productivity']
[True, True, True, True, True, True, True, True, True, True, True, True]


In [27]:
print (ref_noun,falseRef, sep = "\n")

['t', 'lot', 'space', '#', 'beds', 'option', 'Gardening', 'beds', 'way', 'productivity', '#', 'investment', '#', 'materials', 'beds', 'soil', 'compost', 'investment', '#', 'productivity']
[False, False, True, True, True, False, False, True, True, True, True, True, True, True, True, True, True, True, True, True]


In [28]:
def compress(nr,Fr,M):
    x=0
    nr_hat=[]
    while(x< len(Fr)):
        if (Fr[x]==False and x==0):
            for j in range(x+1, len(Fr)):
                if(Fr[j]==False):
                    x = j
        elif (Fr[x]==True):
            nr_hat.append(nr[x])
        elif (Fr[x]==False):
            for j in range(x+1, len(Fr)):
                if(Fr[j]==False):
                    x = j
            nr_hat.append(" ")
        x = x+1

    return nr_hat

In [29]:
def removeContinuoustrue(nr,Fr,M):
    x=0
    pp = 0
    i = 0
    while(i<len(Fr)):
        if Fr[i]==True:
            x=0
            x = x+len(nr[i])
            for j in range(i+1,len(Fr)):
                if Fr[j]==True:
                    x = x+ len(nr[j])
                    pp = j
                else:
                    break

            if (pp==len(Fr)-1):
                break
            if (x<M):
                for k in range(i,pp+1):
                    Fr[k]=False
                i = pp
        i = i+1
    return Fr, nr

In [30]:
def compare(nr_hat,nt_hat,threshold):
    plagText=[]
    for i in range(0,len(nr_hat)+1):
        x = len(nr_hat)-i
        y = 0
        res = np.ones(i) * -1
        while(x<len(nr_hat) and y<i and y<len(nt_hat)):
            if(nr_hat[x]=='#' and nt_hat[y]=='#'):
                res[y]= 0
            elif (nr_hat[x]=='#'):
                res[y] = len(nt_hat[y])
            elif (nt_hat[y] =='#'):
                res[y] = len(nr_hat[x])
            elif (nr_hat[x]==nt_hat[y]):
                res[y] = len(nr_hat[x])
            
            y +=1
            x +=1
        
        x=0
        while x<len(res):
            start = -1
            end = -1
            temp=0
            if (res[x] >=0) :
                start = x
                end=x
                temp+=res[x]
      
                for k in range(x+1,len(res)):
                    if res[k]>=0 :
                        temp+=res[k]
                        end=k
                    else :
                        x=k
                        break
            if (temp> threshold) :
                plagText.append([nr_hat[start:end],temp])
      
            x+=1

    return plagText 

In [31]:
def plagRate(l,l0):
    rate = (l-l0)/l
    return rate

In [39]:
def detectSimilarity(target,references):
    nt = findNouns(target)
    nr = findNouns(references)
    shared = sharedWords(nt,nr)
    mt = findsharedAmount(nt,shared) 
    mr = findsharedAmount(nr,shared)
    threshold = defthreshold(shared)
    threshold = thres(nt,nr,threshold)
    check= mt>=threshold or mr >=threshold
    print(check)
  
    Ft = [False]*len(nt)
    Fr = [False]*len(nr)
  
    Ft = generateflags(Ft,nt,shared)
    Fr = generateflags(Fr,nr,shared)
  
    nt,Ft = findIsolatedFalse(nt,Ft)
    nr,Fr = findIsolatedFalse(nr,Fr)
  
    Fr1,nr_hat = removeContinuoustrue(nr.copy(),Fr.copy(),threshold)
    Ft1,nt_hat = removeContinuoustrue(nt.copy(),Ft.copy(),threshold)
  
    nr_hat = compress(nr_hat,Fr1,threshold)
    nt_hat = compress(nt_hat,Ft1,threshold)
  
    plagiarised = compare(nr_hat, nt_hat, threshold)
  
    psets = set()
    for i in range(0, len(plagiarised)):
        psets = psets.union(set(plagiarised[i][0]))
  
    difference = set(nt_hat).difference(psets)
    print("Plagiarism can be found at/around these section of words--> {}\n".format(psets))
    l0 = 0 
    l = 0
    for i in nt_hat:
        l = l + len(i)
    for i in difference:
        l0 = l0 + len(i)
    return plagRate(l,l0)

In [40]:
simTargetRef = detectSimilarity(target,references)
simOriginalPlag = detectSimilarity(original,plag)

True
Plagiarism can be found at/around these section of words--> {'productivity', 'beds', 'investment', '#', 'materials', 'compost', 'soil', 'way'}

True
Plagiarism can be found at/around these section of words--> {'bankruptcy', 'law', 'system', 'courts', 'specialty', 'family'}



In [42]:
print (f"Plagiarism percent between Target and Reference paragraph : {simTargetRef*100} %")
print (f"Plagiarism percent between Original and Plagiarism paragraph : {simOriginalPlag*100} %")

Plagiarism percent between Target and Reference paragraph : 93.82716049382715 %
Plagiarism percent between Original and Plagiarism paragraph : 74.28571428571429 %
